# Install dependencies, set up folder structure, import necessary librairies, generate data.yaml file

In [ ]:
!pip install albumentations
!pip install opencv-python
!pip install ultralytics
!mkdir data
!mkdir data/train
!mkdir data/test
!mkdir data/val
!mkdir data/train/images
!mkdir data/test/images
!mkdir data/val/images
!mkdir data/train/labels
!mkdir data/test/labels
!mkdir data/val/labels
!rm -r sample_data

In [ ]:
import os, shutil
from os import listdir
from os.path import isfile, join
import cv2
import albumentations as A
import copy
import random
import numpy as np
from ultralytics import YOLO
import torch
import shutil
from google.colab import drive
drive.mount('/content/drive')

In [3]:
with open('data.yaml', 'w+') as f:
    f.write("train: /content/data/train/images\n")
    f.write("test: /content/data/test/images\n")
    f.write("val: /content/data/val/images\n")
    f.write("nc: 1\n")
    f.write('names: ["Lane Marker"]')

# Train YOLO model with augmented data

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
#Ensure this prints true and a number > 0, if not make sure you set hardware accelerator to GPU in Edit > Notebook Settings > Hardware Accelerator
!rm -r runs/segment/train*
!mkdir runs
!mkdir runs/segment
!mkdir runs/segment/train

In [ ]:
#UNCOMMENT TO START FROM SCRATCH
model = YOLO("yolov8n-seg.pt")  # load a pretrained model (recommended for training)

# CONTINUE TRAINING
#model = YOLO("/content/drive/My Drive/AUV_model.pt") #load a previous model in case training interrupts

# Train the model in increments
epoch_increments = 10
i = 2
while True:
  model.train(data="data.yaml", epochs=epoch_increments, device=0, batch=16, degrees=360, flipud=0.5, fliplr=0.5, perspective=0.001, translate=0.5, scale=0.5, pretrained=True, task='segment')  # train the model
  model.val()
  shutil.copyfile("runs/detect/train" + str(i) + "/weights/best.pt", "/content/drive/My Drive/AUV_model_seg_" + str(i) + ".pt")
  i = i + 1

# Predict on image with model

In [ ]:
img_path = "data/augmented/train/images/img3.png"
img = cv2.imread(img_path)
# Load a model
model = YOLO("runs/detect/train2/weights/best.pt")  # load a model
# Use the model
pred = model(img)
for results in pred:
    box = results.boxes
    print(box)
    print("\nprediction:" + str(list(box.xywh)))
    print("\nprediction:" + str(list(box.conf)))
    print("\nprediction:" + str(list(box.cls)))